# Лабораторная работа №8 (Проведение исследований моделями обнаружения и распознавания объектов)
## Выполнил студент группы М8О-406Б-21, Орусский В.Р.

Был использован датасет [VisDrone](https://www.kaggle.com/datasets/evilspirit05/visdrone)

Набор данных содержит изображения и видео с дронов, позволяющие детектировать объекты с высоты. Очень полезно для инспекции больших пространств с высоты, например при патрулировании больших территорий заводов, фабрик или других охраняемых зон. Так же в нынешнее время полезно для ведения боевых действий.

Импорт зависимостей и библиотек

In [7]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import torch
import warnings
from ultralytics import YOLO
import os
import yaml

# Отключаем только несущественные предупреждения
warnings.filterwarnings("ignore", category=UserWarning)

# Установка ultralytics
try:
    from ultralytics import YOLO
except ImportError:
    print("Устанавливаем ultralytics...")
    os.system("pip install ultralytics")
    from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\slava\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import kagglehub

path = kagglehub.dataset_download('evilspirit05/visdrone')

print(path)

C:\Users\slava\.cache\kagglehub\datasets\evilspirit05\visdrone\versions\14


In [ ]:
TRAIN_IMAGES = os.path.join(DATA_DIR, 'VisDrone2019-DET-train/images')
TRAIN_ANNOTATIONS = os.path.join(DATA_DIR, 'VisDrone2019-DET-train/annotations')
TRAIN_LABELS = os.path.join(DATA_DIR, 'VisDrone2019-DET-train/labels')
VAL_IMAGES = os.path.join(DATA_DIR, 'VisDrone2019-DET-val/images')
VAL_ANNOTATIONS = os.path.join(DATA_DIR, 'VisDrone2019-DET-val/annotations')
VAL_LABELS = os.path.join(DATA_DIR, 'VisDrone2019-DET-val/labels')

# Создание директорий для YOLO меток
os.makedirs(TRAIN_LABELS, exist_ok=True)
os.makedirs(VAL_LABELS, exist_ok=True)

train_images_count = len(os.listdir(TRAIN_IMAGES)) if os.path.exists(TRAIN_IMAGES) else 0
train_labels_count = len(os.listdir(TRAIN_ANNOTATIONS)) if os.path.exists(TRAIN_ANNOTATIONS) else 0
val_images_count = len(os.listdir(VAL_IMAGES)) if os.path.exists(VAL_IMAGES) else 0
val_labels_count = len(os.listdir(VAL_ANNOTATIONS)) if os.path.exists(VAL_ANNOTATIONS) else 0

Набор данных

In [12]:
print(f"Тренировочные изображения: {train_images_count}, метки: {train_labels_count}")
print(f"Валидационные изображения: {val_images_count}, метки: {val_labels_count}")

Тренировочные изображения: 6471, метки: 6471
Валидационные изображения: 548, метки: 548


Создание YOLO меток

In [ ]:
def convert_bbox_to_YOLO_format(image_path, annotation_path, output_label_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise Exception(f"Failed to read image: {image_path}")
        h, w = image.shape[:2]
        with open(annotation_path, 'r') as f:
            lines = f.readlines()
        yolo_annotations = []
        for line in lines:
            data = line.strip().split(',')
            if len(data) < 6:
                continue
            bbox_left, bbox_top, bbox_width, bbox_height, _, class_id = map(float, data[:6])
            class_id = int(class_id) - 1  # VisDrone классы начинаются с 1
            if class_id < 0 or class_id >= 10:
                continue
            x_cen = (bbox_left + bbox_width / 2) / w
            y_cen = (bbox_top + bbox_height / 2) / h
            box_w = bbox_width / w
            box_h = bbox_height / h
            yolo_annotations.append(f"{class_id} {x_cen:.6f} {y_cen:.6f} {box_w:.6f} {box_h:.6f}")
        with open(output_label_path, 'w') as f:
            f.write('\n'.join(yolo_annotations) + '\n')
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

Конвертация аннотаций

In [ ]:
for ann_file in tqdm(os.listdir(TRAIN_ANNOTATIONS), desc="Converting train annotations"):
    if ann_file.endswith('.txt'):
        img_file = ann_file.replace('.txt', '.jpg')
        img_path = os.path.join(TRAIN_IMAGES, img_file)
        ann_path = os.path.join(TRAIN_ANNOTATIONS, ann_file)
        label_path = os.path.join(TRAIN_LABELS, ann_file)
        if os.path.exists(img_path):
            convert_bbox_to_YOLO_format(img_path, ann_path, label_path)

for ann_file in tqdm(os.listdir(VAL_ANNOTATIONS), desc="Converting val annotations"):
    if ann_file.endswith('.txt'):
        img_file = ann_file.replace('.txt', '.jpg')
        img_path = os.path.join(VAL_IMAGES, img_file)
        ann_path = os.path.join(VAL_ANNOTATIONS, ann_file)
        label_path = os.path.join(VAL_LABELS, ann_file)
        if os.path.exists(img_path):
            convert_bbox_to_YOLO_format(img_path, ann_path, label_path)

YAML-конфигурация для YOLO


In [ ]:
data_config = {
    'path': DATA_DIR,
    'train': 'VisDrone2019-DET-train/images',
    'val': 'VisDrone2019-DET-val/images',
    'nc': 10,
    'names': ['pedestrian', 'people', 'bicycle', 'car', 'van', 'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor']
}
with open('visdrone.yaml', 'w') as f:
    yaml.dump(data_config, f)

Бейзлайн (YOLOv11n)

In [ ]:
model = YOLO('yolov11n.pt')
model.train(data='visdrone.yaml', epochs=10, imgsz=640, batch=8, device=0)
metrics = model.val()
baseline_metrics = {
    'mAP@50': metrics.box.map50,
    'mAP@50:95': metrics.box.map,
    'Precision': metrics.box.p,
    'Recall': metrics.box.r
}
print(f"Бейзлайн: mAP@50: {baseline_metrics['mAP@50']:.4f}, mAP@50:95: {baseline_metrics['mAP@50:95']:.4f}")

Результат

In [ ]:
img_path = os.path.join(VAL_IMAGES, os.listdir(VAL_IMAGES)[0])
img = cv2.imread(img_path)
results = model.predict(img_path)
for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    classes = r.boxes.cls.cpu().numpy()
    for box, cls in zip(boxes, classes):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, data_config['names'][int(cls)], (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

Улучшенный бейзлайн (YOLOv11s с аугментациями)

In [ ]:
improved_model = YOLO('yolov11s.pt')
improved_model.train(
    data='visdrone.yaml',
    epochs=10,
    imgsz=640,
    batch=8,
    device=0,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, flipud=0.5  # Аугментации
)
improved_metrics = improved_model.val()
improved_metrics_dict = {
    'mAP@50': improved_metrics.box.map50,
    'mAP@50:95': improved_metrics.box.map,
    'Precision': improved_metrics.box.p,
    'Recall': improved_metrics.box.r
}
print(f"Улучшенный бейзлайн: mAP@50: {improved_metrics_dict['mAP@50']:.4f}, mAP@50:95: {improved_metrics_dict['mAP@50:95']:.4f}")
print(f"Сравнение: ΔmAP@50: {(improved_metrics_dict['mAP@50'] - baseline_metrics['mAP@50']):.4f}")


Собственная реализация

In [ ]:
class SimpleYOLO(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleYOLO, self).__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.head = nn.Conv2d(32, num_classes * 5, 1)  # 5: class_id, x, y, w, h

    def forward(self, x):
        x = self.backbone(x)
        return self.head(x)